## NEVER LET A WINNING TRADE TURN INTO A LOSS!


In [3]:
from IPython.core.display import HTML
import logging
from IPython.display import YouTubeVideo



HTML("""
<style>
.container { width:100% !important; }
</style>
""")

### Connect to IB

In [4]:
from ib_async import *
util.startLoop()
ib = IB()
ib.connect(port=4004,clientId=0)

<IB connected to 127.0.0.1:4004 clientId=0>

### Get TSLA Market Data

In [5]:
tsla_contract = Stock(symbol='TSLA',exchange='SMART',currency='USD')
ib.qualifyContracts(tsla_contract)
tsla_contract

Stock(conId=76792991, symbol='TSLA', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='TSLA', tradingClass='NMS')

In [6]:
tsla_ticker = ib.reqMktData(tsla_contract,"",False,False)

In [8]:
tsla_ticker.marketPrice()

174.64999999999998

* limitPrice = 174.6
* takeProfit = 175.6
* stopLoss = NOT ACTIVE UNLESS TRADE GOES IN YOUR FAVOR. THEN MOVE TO BREAK EVEN

### Set up a bracket order

In [10]:
limit_price = tsla_ticker.marketPrice()-0.05
take_profit_price = limit_price+1
stop_price = limit_price+0.5 
# Note this will trigger right away so it is a bad idea, but we can put a price condition on it

bracket_order = ib.bracketOrder(action="BUY",quantity=100,limitPrice=limit_price,takeProfitPrice=take_profit_price,stopLossPrice=stop_price)

In [13]:
price_to_activate_stop_order = limit_price + (take_profit_price-limit_price)*3/4
limit_price,price_to_activate_stop_order,stop_price

(174.66, 175.41, 175.16)

In [15]:
price_condition = PriceCondition(conId=tsla_contract.conId,price=price_to_activate_stop_order,isMore=True)


In [16]:
# To store our bracket Trade
bracket_trade = {}
for order in bracket_order:
    if order.orderType == 'STP': # For the stop order add the price condition!
        order.conditions.append(price_condition)
    bracket_trade[(order.action,order.orderType)]=ib.placeOrder(tsla_contract,order)
    

Error 148, reqId 4819: Conditional submission of orders is supported for Limit, Market, MidPrice, Relative and Snap order types only.  Conditional cancelation of orders is supported for Limit and MidPrice order types only.
Canceled order: Trade(contract=Stock(conId=76792991, symbol='TSLA', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='TSLA', tradingClass='NMS'), order=StopOrder(orderId=4819, action='SELL', totalQuantity=100, auxPrice=175.16, parentId=4817, conditions=[PriceCondition(condType=1, conjunction='a', isMore=True, price=175.41, conId=76792991, exch='', triggerMethod=0)]), orderStatus=OrderStatus(orderId=4819, status='Cancelled', filled=0.0, remaining=0.0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2024, 5, 20, 17, 7, 49, 36810, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0), TradeLogEntry(time=datetime.dateti

### Cancel Order

In [18]:
for order in ib.openOrders():
    ib.cancelOrder(order)

Error 10147, reqId 4818: OrderId 4818 that needs to be cancelled is not found.


#### Set Initial stop price to dummy amount...

In [61]:
def place_order_with_with_ticker_callback(contract,order,parent_trade,
                                          price_to_activate_stop_order,stop_price):
    ticker = ib.ticker(contract)
    stop_trade:Trade = ib.placeOrder(contract,order)
    def on_ticker_update(ticker:Ticker):
        nonlocal stop_trade
        print("CALLING ON TICKER UPDATE",ticker.marketPrice(),price_to_activate_stop_order,parent_trade.orderStatus.status,stop_trade.order.auxPrice)
        # This doesn't necessarily handle partial fills well, but it is what I got for now :)
        if ticker.marketPrice()>=price_to_activate_stop_order and parent_trade.orderStatus.status=='Filled' and parent_trade.orderStatus.remaining==0:
            print("UPDATING STOP PRICE")
            order = stop_trade.order
            order.auxPrice = stop_price
            stop_trade=ib.placeOrder(contract,order)
            # Detach callback
            ticker.updateEvent-=on_ticker_update
    ticker.updateEvent+=on_ticker_update
    # This is probably not needed, but cancel this callback if the order gets cancelled...
    def on_order_cancel(trade):
        ticker.updateEvent-=on_ticker_update
    def print_order_for_debugging(trade):
        print("Stop Loss has Been Updated")
        print(trade.order)
    stop_trade.cancelEvent+=on_order_cancel
    stop_trade.modifyEvent+=print_order_for_debugging
    
    return stop_trade
        

In [62]:
limit_price=tsla_ticker.marketPrice()
initial_stop_price = limit_price-50
take_profit_price = limit_price+1
stop_price = limit_price+0.05

price_to_activate_stop_order = limit_price + (take_profit_price-limit_price)*1/8

limit_price,take_profit,stop_price,price_to_activate_stop_order
bracket_order = ib.bracketOrder(action="BUY",quantity=100,limitPrice=limit_price,takeProfitPrice=take_profit_price,stopLossPrice=initial_stop_price)

In [63]:
bracket_order,limit_price,take_profit_price

(BracketOrder(parent=LimitOrder(orderId=4841, action='BUY', totalQuantity=100, lmtPrice=174.18, transmit=False), takeProfit=LimitOrder(orderId=4842, action='SELL', totalQuantity=100, lmtPrice=175.18, transmit=False, parentId=4841), stopLoss=StopOrder(orderId=4843, action='SELL', totalQuantity=100, auxPrice=124.18, parentId=4841)),
 174.18,
 175.18)

In [64]:
price_to_activate_stop_order = limit_price + (take_profit_price-limit_price)*1/8
# To store our bracket Trade
bracket_trade = {}
for order in bracket_order:
    if order.orderType == 'STP': # For the stop order add the price condition!
        parent_trade = bracket_trade[('BUY','LMT')]
        bracket_trade[(order.action,order.orderType)]=place_order_with_with_ticker_callback(tsla_contract,order,parent_trade,price_to_activate_stop_order,stop_price)
    else:
        bracket_trade[(order.action,order.orderType)]=ib.placeOrder(tsla_contract,order)
        
    

CALLING ON TICKER UPDATE 174.29 174.305 PendingSubmit 124.18
CALLING ON TICKER UPDATE 174.28 174.305 PendingSubmit 124.18
CALLING ON TICKER UPDATE 174.28 174.305 Submitted 124.18
CALLING ON TICKER UPDATE 174.29 174.305 Submitted 124.18
CALLING ON TICKER UPDATE 174.29 174.305 Submitted 124.18
CALLING ON TICKER UPDATE 174.29 174.305 Submitted 124.18
CALLING ON TICKER UPDATE 174.28 174.305 Submitted 124.18
CALLING ON TICKER UPDATE 174.29 174.305 Submitted 124.18
CALLING ON TICKER UPDATE 174.3 174.305 Submitted 124.18
CALLING ON TICKER UPDATE 174.28 174.305 Submitted 124.18
CALLING ON TICKER UPDATE 174.265 174.305 Submitted 124.18
CALLING ON TICKER UPDATE 174.255 174.305 Submitted 124.18
CALLING ON TICKER UPDATE 174.27 174.305 Submitted 124.18
CALLING ON TICKER UPDATE 174.25 174.305 Submitted 124.18
CALLING ON TICKER UPDATE 174.25 174.305 Submitted 124.18
CALLING ON TICKER UPDATE 174.26 174.305 Submitted 124.18
CALLING ON TICKER UPDATE 174.235 174.305 Submitted 124.18
CALLING ON TICKER UPD

In [73]:
ib.openOrders()

[]

In [45]:
ib.openOrders()

CALLING ON TICKER UPDATE 174.78 174.855 Filled


[LimitOrder(orderId=4830, permId=707059279, action='SELL', totalQuantity=100.0, lmtPrice=175.73, auxPrice=0.0, transmit=False, parentId=4829),
 StopOrder(orderId=4831, permId=707059280, action='SELL', totalQuantity=100.0, lmtPrice=0.0, auxPrice=124.73, parentId=4829)]

CALLING ON TICKER UPDATE 174.78 174.855 Filled
CALLING ON TICKER UPDATE 174.78 174.855 Filled
CALLING ON TICKER UPDATE 174.77 174.855 Filled
CALLING ON TICKER UPDATE 174.77 174.855 Filled
CALLING ON TICKER UPDATE 174.77 174.855 Filled
CALLING ON TICKER UPDATE 174.77 174.855 Filled
CALLING ON TICKER UPDATE 174.77 174.855 Filled
CALLING ON TICKER UPDATE 174.77 174.855 Filled
CALLING ON TICKER UPDATE 174.76 174.855 Filled
CALLING ON TICKER UPDATE 174.76 174.855 Filled
CALLING ON TICKER UPDATE 174.755 174.855 Filled
CALLING ON TICKER UPDATE 174.74 174.855 Filled
CALLING ON TICKER UPDATE 174.75 174.855 Filled
CALLING ON TICKER UPDATE 174.75 174.855 Filled
CALLING ON TICKER UPDATE 174.73 174.855 Filled
CALLING ON TICKER UPDATE 174.73 174.855 Filled
CALLING ON TICKER UPDATE 174.73 174.855 Filled
CALLING ON TICKER UPDATE 174.73 174.855 Filled
CALLING ON TICKER UPDATE 174.73 174.855 Filled
CALLING ON TICKER UPDATE 174.73 174.855 Filled
CALLING ON TICKER UPDATE 174.7 174.855 Filled
CALLING ON TI

In [60]:
for order in ib.openOrders():
    ib.cancelOrder(order)

CALLING ON TICKER UPDATE 174.2 174.315 Filled 0.0


### Look at our fills

In [74]:
util.df(ib.fills())

,contract,execution,commissionReport,time
0,"Stock(conId=76792991, symbol='TSLA', right='?'...","Execution(execId='00025b46.664acc18.01.01', ti...",CommissionReport(execId='00025b46.664acc18.01....,2024-05-20 10:59:23+00:00
1,"Stock(conId=76792991, symbol='TSLA', right='?'...","Execution(execId='00025b46.664acc1a.01.01', ti...",CommissionReport(execId='00025b46.664acc1a.01....,2024-05-20 10:59:31+00:00
2,"Stock(conId=76792991, symbol='TSLA', right='?'...","Execution(execId='00025b46.664acc1b.01.01', ti...",CommissionReport(execId='00025b46.664acc1b.01....,2024-05-20 10:59:38+00:00
3,"Stock(conId=76792991, symbol='TSLA', right='?'...","Execution(execId='00025b46.664acc3b.01.01', ti...",CommissionReport(execId='00025b46.664acc3b.01....,2024-05-20 11:00:56+00:00
4,"Stock(conId=76792991, symbol='TSLA', right='?'...","Execution(execId='00025b46.664acc3c.01.01', ti...",CommissionReport(execId='00025b46.664acc3c.01....,2024-05-20 11:00:56+00:00
5,"Stock(conId=76792991, symbol='TSLA', right='?'...","Execution(execId='00025b46.664acc41.01.01', ti...",CommissionReport(execId='00025b46.664acc41.01....,2024-05-20 11:01:59+00:00
6,"Stock(conId=76792991, symbol='TSLA', right='?'...","Execution(execId='00025b46.664acd1b.01.01', ti...",CommissionReport(execId='00025b46.664acd1b.01....,2024-05-20 11:10:29+00:00
7,"Stock(conId=76792991, symbol='TSLA', right='?'...","Execution(execId='00025b46.664acd1c.01.01', ti...",CommissionReport(execId='00025b46.664acd1c.01....,2024-05-20 11:10:36+00:00
8,"Stock(conId=76792991, symbol='TSLA', right='?'...","Execution(execId='00025b46.664acd24.01.01', ti...",CommissionReport(execId='00025b46.664acd24.01....,2024-05-20 11:12:28+00:00
9,"Stock(conId=76792991, symbol='TSLA', right='?'...","Execution(execId='00025b46.664acd2c.01.01', ti...",CommissionReport(execId='00025b46.664acd2c.01....,2024-05-20 11:15:37+00:00


In [79]:
util.df(ib.fills()).execution.apply(lambda l:l.price)

0     178.05
1     178.05
2     178.00
3     177.90
4     177.90
5     177.93
6     177.90
7     177.90
8     177.97
9     177.98
10      0.82
11    176.00
12    174.32
13    174.27
14    175.14
15    174.64
16    174.96
17    174.39
18    174.12
19    174.42
20    174.76
21    175.06
22    175.01
23    175.00
24    174.93
25    175.23
26    175.40
27    174.95
28    174.85
29    174.68
30    174.41
31    174.39
32    174.37
33    174.68
34    174.45
35    174.54
36    174.84
37    174.61
38    174.65
39    174.38
40    174.06
41    174.11
42    173.69
43    174.19
44    174.15
45    174.16
46    174.03
47    173.91
48    174.29
49    174.37
50    174.73
51    174.57
52    174.27
53    174.17
54    174.18
55    174.21
Name: execution, dtype: float64